In [1]:
import pandas as pd

In [2]:
emails_data = pd.read_csv('./all_emails_meetings.csv')
emails_data.head()

,Date,From,To,Subject,Body,Time,Class
0,2000-11-16,frozenset({'enron.announcements@enron.com'}),frozenset({'houston.report@enron.com'}),Weekend Outage for 11-17-00 through 11-20-00,w e e k e n d s y s t e m s a v a i l a b i l ...,10:08:00,Meeting-related
1,2001-05-22,frozenset({'greg.piper@enron.com'}),frozenset({'greg.whalley@enron.com'}),"EnronOnline, etc.",did you talk to dynegy about buying the engine...,12:22:00,Meeting-related
2,2001-07-24,frozenset({'jeff.dasovich@enron.com'}),frozenset({'sgovenar@govadv.com'}),Rumor That The Legislature May Reconvene,bizarre as it sounds weve now heard that they ...,12:02:00,Meeting-related
3,2000-09-25,frozenset({'rosalee.fleming@enron.com'}),frozenset({'joe.hillings@enron.com'}),Re: Selection Of My Successor-My Concern,good morning joe ken is on a cruise so is not ...,03:39:00,Meeting-related
4,1979-12-31,frozenset({'mary.hain@enron.com'}),NaN,Don't forget to register!,to register your palm computingc organizer ele...,16:00:00,Meeting-related


In [3]:
emails_data = emails_data.drop('Class', axis=True)

In [4]:
def clean_frozenset_email(fset):
    if fset is None or pd.isna(fset):
        return None  # Handle None or NaN values
    fset = fset.replace("frozenset","")
    fset = fset.replace("(","")
    fset = fset.replace(")","")
    fset = fset.replace("{","")
    fset = fset.replace("}","")
    fset = fset.split(',')
    return list(fset)

# Apply the function to the 'From' and 'To' columns
emails_data['From'] = emails_data['From'].apply(clean_frozenset_email)
emails_data['To'] = emails_data['To'].apply(clean_frozenset_email)
emails_data.head()

,Date,From,To,Subject,Body,Time
0,2000-11-16,['enron.announcements@enron.com'],['houston.report@enron.com'],Weekend Outage for 11-17-00 through 11-20-00,w e e k e n d s y s t e m s a v a i l a b i l ...,10:08:00
1,2001-05-22,['greg.piper@enron.com'],['greg.whalley@enron.com'],"EnronOnline, etc.",did you talk to dynegy about buying the engine...,12:22:00
2,2001-07-24,['jeff.dasovich@enron.com'],['sgovenar@govadv.com'],Rumor That The Legislature May Reconvene,bizarre as it sounds weve now heard that they ...,12:02:00
3,2000-09-25,['rosalee.fleming@enron.com'],['joe.hillings@enron.com'],Re: Selection Of My Successor-My Concern,good morning joe ken is on a cruise so is not ...,03:39:00
4,1979-12-31,['mary.hain@enron.com'],None,Don't forget to register!,to register your palm computingc organizer ele...,16:00:00


In [5]:
emails_data_shuffled = emails_data.sample(frac=1, random_state=42).reset_index(drop=True)
emails_data_shuffled.head()

,Date,From,To,Subject,Body,Time
0,2000-12-04,['dan.hyvl@enron.com'],['mcesario@ormet.com'],Re: Transaction Agreement,exhibit c and exhibit d mike cesario mcesarioo...,06:14:00
1,2000-12-08,['james.steffes@enron.com'],['kevin.presto@enron.com'],RTO Status 12.8.00,kevin attached is the east power rto status ma...,09:36:00
2,2000-11-27,['susan.scott@enron.com'],['mary.miller@enron.com'],Cal. GIR,for your information the outcome was great for...,01:29:00
3,2000-09-20,['shirley.crenshaw@enron.com'],"['zimin.lu@enron.com', 'vasant.shanbhogue@enr...",Re: Informal Interview with the Enron Research...,fyi i have arranged the following interview sc...,08:47:00
4,2001-10-23,['d..steffes@enron.com'],['jean.ryall@enron.com'],RE:,thanks if you need any additional support pls ...,08:41:42


In [6]:
emails_data_shuffled = emails_data_shuffled[5500:20000]
emails_data_shuffled['Intent and Data'] = 0

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os
os.environ["OPENAI_API_KEY"] = "sk-proj-dfRg3czB5YQH0pGTmsUeT3BlbkFJ0c0ezQMmuQDKsKWhCXHY"

In [10]:
llm_model = "gpt-3.5-turbo-0125"

In [11]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

In [12]:
intent_analysis_prompt = ChatPromptTemplate.from_template(
    """
    
    Intent Analysis: Determine if the email intends to schedule a meeting. Provide a 'Yes' or 'No' response. 
    If 'Yes', provide the details in the specified format.

    Instructions:
    - If the email suggests scheduling a meeting, return details as a JSON array with the specified structure.
    - Ensure all fields are string type and dates and times are formatted as 'dd-mm-yyyy hh:mm'.
    - Use 'start' and 'end' keys for time data.
    - Give me the JSON only if the intent is to schedule a meeting and dont give any json if there is no intent to schedule a meeting

    Output Format:
        'Yes', JSON or 'No'
        
        Below mentioned is the JSON format
        
        'sender': 'email address of the sender',
        'receiver': 'email address of the receiver',
        'subject': 'Subject of the email',
        'description': 'A brief summary of the email',
        'platform or location': 'Platform or Location of the meeting',
        'data': 'additional data in key-value pairs'

    Additional Notes:
    - Do not use 'JSON' or backticks in your response.
    - Ensure the JSON response starts with an opening square brace and is parsable without further processing.
    - Avoid special characters outside the JSON body.
    
    Email:
    {email}
    
    """

)

In [13]:
intent_analysis_prompt = LLMChain(llm=llm, prompt=intent_analysis_prompt)

In [14]:
input_data = emails_data_shuffled.iloc[1]
result = intent_analysis_prompt.run({"email": input_data})

In [15]:
result

'No'

In [16]:
import time

In [17]:
def get_intent_and_data(data, x):
    for i in range(data.shape[0]):
        input_data = data.iloc[i]
        attempts = 0
        success = False
        while not success and attempts < 3:  # Retry up to 3 times
            try:
                result = intent_analysis_prompt.run({"email": input_data})
                data.iloc[i, -1] = result
                success = True  # If API call was successful
            except Exception as e:
                print(f"API error at index {i}, attempt {attempts + 1}: {e}")
                time.sleep(30 * attempts)  # Exponential backoff
                attempts += 1
        if not success:
            data.iloc[i, -1] = 'Error'  # Mark as error if all retries fail

    # Save the results to CSV
    data.to_csv(f'./intent/email_intent{x}.csv', index=False)


In [18]:
n = 40
step = 500

for i in range(n):
    if (i + 1) * step <= emails_data_shuffled.shape[0]:  # Ensure the last batch does not exceed the data size
        batch_data = emails_data_shuffled.iloc[i*step : (i+1)*step]
        get_intent_and_data(batch_data, i)
        print(f"Batch {i} processed")
    else:
        batch_data = emails_data_shuffled.iloc[i*step :]
        get_intent_and_data(batch_data, i)
        print(f"Final batch {i} processed")

Batch 0 processed
Batch 1 processed
Batch 2 processed
Batch 3 processed
Batch 4 processed
Batch 5 processed
Batch 6 processed
Batch 7 processed
Batch 8 processed
Batch 9 processed
Batch 10 processed
Batch 11 processed
Batch 12 processed


Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-

API error at index 9, attempt 1: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.


Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..


API error at index 10, attempt 1: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.


...

Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..


API error at index 409, attempt 1: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.


Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..
Retrying langchain_community.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more..


API error at index 410, attempt 1: Rate limit reached for gpt-3.5-turbo-0125 in organization org-gwFi9j1xfIGl3COmN1Nukl6n on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.


KeyboardInterrupt: 